# Time Series Anomaly Detection

In [1]:
#module imports
import wrangle as w

# from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

## Acquire

In [2]:
df = w.acquire_logs()
df.head()

,date,time,name,program_id,cohort_id,user_id,path,ip,start_date,end_date
0,2018-01-26,09:55:03,Hampton,1,8.0,1,/,97.105.19.61,2015-09-22,2016-02-06
1,2018-01-26,09:56:02,Hampton,1,8.0,1,java-ii,97.105.19.61,2015-09-22,2016-02-06
2,2018-01-26,09:56:05,Hampton,1,8.0,1,java-ii/object-oriented-programming,97.105.19.61,2015-09-22,2016-02-06
3,2018-01-26,09:56:06,Hampton,1,8.0,1,slides/object_oriented_programming,97.105.19.61,2015-09-22,2016-02-06
4,2018-01-26,09:56:24,Teddy,2,22.0,2,javascript-i/conditionals,97.105.19.61,2018-01-08,2018-05-17


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847330 entries, 0 to 847329
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        847330 non-null  object 
 1   time        847330 non-null  object 
 2   name        847330 non-null  object 
 3   program_id  847330 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   user_id     847330 non-null  int64  
 6   path        847329 non-null  object 
 7   ip          847330 non-null  object 
 8   start_date  847330 non-null  object 
 9   end_date    847330 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 64.6+ MB


In [4]:
df.shape

(847330, 10)

## Prepare

In [ ]:
#change time/date from object to datetime
df.index = pd.to_datetime(df.date + " " + df.time)

In [ ]:
df.head()

In [ ]:
#drop date and time columns
df.drop(columns=['date', 'time'], inplace=True)

In [ ]:
df.head()

In [ ]:
#add day of week and month columns for exploration
df['day_of_week'] = df.index.day_name()
df['month'] = df.index.month_name()

In [ ]:
df.head()

In [ ]:
df.program_id.value_counts()

In [ ]:
def prepare_logs():
    '''
    This function will acquire the dataset from acquire_logs
    and prepare for exploration by:
    - reformatting date/time columns and merging them
    - resetting date/time as index
    - adding weekday and month columns
    - renaming columns
    '''
    # Grabs dataframe from mysql and assigns it to a variable
    df = w.acquire_logs()
    # Reformat date and time
    df.index = pd.to_datetime(df.date + " " + df.time)
    #drop date and time columns
    df.drop(columns=['date', 'time'], inplace=True)
    # Replace program_id values with names of program
    df['program_id'] = df.program_id.replace({1: 'full stack php', 2: 'full stack java', 3:'data science', 4:'front end' })
    #add day of week and month columns for exploration
    df['day_of_week'] = df.index.day_name()
    df['month'] = df.index.month_name()

    return df

In [ ]:
import wrangle as w

In [6]:
df = w.prepare_logs()

In [8]:
df.head()

,name,program_id,cohort_id,user_id,path,ip,start_date,end_date,day_of_week,month
2018-01-26 09:55:03,Hampton,full stack php,8.0,1,/,97.105.19.61,2015-09-22,2016-02-06,Friday,January
2018-01-26 09:56:02,Hampton,full stack php,8.0,1,java-ii,97.105.19.61,2015-09-22,2016-02-06,Friday,January
2018-01-26 09:56:05,Hampton,full stack php,8.0,1,java-ii/object-oriented-programming,97.105.19.61,2015-09-22,2016-02-06,Friday,January
2018-01-26 09:56:06,Hampton,full stack php,8.0,1,slides/object_oriented_programming,97.105.19.61,2015-09-22,2016-02-06,Friday,January
2018-01-26 09:56:24,Teddy,full stack java,22.0,2,javascript-i/conditionals,97.105.19.61,2018-01-08,2018-05-17,Friday,January
